In [ ]:
# default_exp dataset

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script

In [ ]:
#export
from fastai.vision.all import *
import skimage.io as skio
import warnings
warnings.filterwarnings("ignore")
import SSLRS.utils as utils


In [ ]:
#export
def open_tif(fn, cls=torch.Tensor):
    im = skio.imread(str(fn))/10000
    im = im.transpose(1,2,0).astype('float32')
    return im
class MSTensorImage(TensorImage):   
    @classmethod
    def create(cls, data:(Path,str,ndarray), chnls=None):
        
        if isinstance(data, Path) or isinstance(data, str):
            if str(data).endswith('tif'): im = open_tif(fn=data,cls=torch.Tensor)

        elif isinstance(data, ndarray): 
            im = torch.from_numpy(data)
        else:
            im = data
        
        return im

In [ ]:
#export
BAND_STATS = {
            'S2':{
                'mean': {
                    'B01': 340.76769064,
                    'B02': 429.9430203,
                    'B03': 614.21682446,
                    'B04': 590.23569706,
                    'B05': 950.68368468,
                    'B06': 1792.46290469,
                    'B07': 2075.46795189,
                    'B08': 2218.94553375,
                    'B8A': 2266.46036911,
                    'B09': 2246.0605464,
                    'B11': 1594.42694882,
                    'B12': 1009.32729131
                },
                'std': {
                    'B01': 554.81258967,
                    'B02': 572.41639287,
                    'B03': 582.87945694,
                    'B04': 675.88746967,
                    'B05': 729.89827633,
                    'B06': 1096.01480586,
                    'B07': 1273.45393088,
                    'B08': 1365.45589904,
                    'B8A': 1356.13789355,
                    'B09': 1302.3292881,
                    'B11': 1079.19066363,
                    'B12': 818.86747235
                }
            },
            'S1': {
                'mean': {
                    'VV': -12.619993741972035,
                    'VH': -19.29044597721542,
                    'VV/VH': 0.6525036195871579,
                },
                'std': {
                    'VV': 5.115911777546365,
                    'VH': 5.464428464912864,
                    'VV/VH': 30.75264076801808,
                },
                'min': {
                    'VV': -74.33214569091797,
                    'VH': -75.11137390136719,
                    'R': 3.21E-2
                },
                'max': {
                    'VV': 34.60696029663086,
                    'VH': 33.59768295288086,
                    'R': 1.08
                }
            }
        }


In [ ]:
#export
bands=['B02','B03', 'B04', 'B05','B06', 'B07', 'B11', 'B08','B8A', 'B12']
means=[BAND_STATS['S2']['mean'][band]/10000 for band in bands]
stds=[BAND_STATS['S2']['std'][band]/10000 for band in bands]

In [ ]:
#export
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from albumentations.core.transforms_interface import ImageOnlyTransform
import random
class Channelaug(ImageOnlyTransform):

    def __init__(self, always_apply=False, p=0.5):
        super(Channelaug, self).__init__(always_apply, p)

    def apply(self, img, **params):
        temp=img[:,:,random.randint(0,9),np.newaxis]
        return np.repeat(temp,10,axis=2)


trainaug=[]
# first gobal crop
globalaug1 = A.Compose([Channelaug(p=0.2),A.HorizontalFlip(p=0.5),
                      A.ShiftScaleRotate(p=.5),
                      A.RandomResizedCrop(120,120,scale=(0.4, 1.),always_apply=True),
                      A.GaussianBlur(p=1.0),
#                       A.Solarize(threshold=0.5),
                      A.Normalize(mean=means,std=stds,max_pixel_value=1.0),
                         ToTensorV2()]                    
                    )

# second global crop
globalaug2 = A.Compose([Channelaug(p=0.2),A.HorizontalFlip(p=0.5),
                      A.ShiftScaleRotate(p=.5),
                      A.RandomResizedCrop(120,120,scale=(0.4, 1.),always_apply=True),
                      A.GaussianBlur(p=0.1),
                      A.Solarize(threshold=0.5,p=0.2),
                      A.Normalize(mean=means,std=stds,max_pixel_value=1.0),
                         ToTensorV2()]                    
                    )
# transformation for the local small crops

locaaug = A.Compose([Channelaug(p=0.2),A.HorizontalFlip(p=0.5),
                      A.ShiftScaleRotate(p=.5),
                      A.RandomResizedCrop(56,56,scale=(0.05, 0.4),always_apply=True),
                      A.GaussianBlur(p=0.5),
                      A.Normalize(mean=means,std=stds,max_pixel_value=1.0),
                         ToTensorV2()]                    
                    )


trainaug.append(globalaug1)
trainaug.append(globalaug2)
for _ in range(6):
    trainaug.append(locaaug)

val_pipe = A.Compose([
    A.Normalize(mean=means,std=stds,max_pixel_value=1.0),
                         ToTensorV2()]                    
                    )

class TrainTransform(ItemTransform):
    split_idx = 0
    def __init__(self, aug,split=0): 
        self.aug = aug
#         self.split_idx = split
    def encodes(self, x):
        result=[]
        for i in range(len(self.aug)):
            result.append(self.aug[i](image=x[0])['image'])
        return result, x[1]
class ValTransform(ItemTransform):
    split_idx = 1
    def __init__(self, aug,split=0): 
        self.aug = aug
#         self.split_idx = split
    def encodes(self, x):
        aug = self.aug(image=x[0])
#         print(torch.cat((aug['image0'],aug['image1']),axis=0).shape)
        return aug['image'], x[1]

# Create our class with this aug_pipe
aug = TrainTransform(trainaug)
aug2=ValTransform(val_pipe)

NameError: name 'means' is not defined

In [ ]:
#export
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(MSTensorImage.create)),CategoryBlock),
                   splitter=ColSplitter('Isval'),
                   get_x=ColReader('fname'),
                   get_y=ColReader('labels'),
                   item_tfms=[aug,aug2]
              )

In [ ]:
df=pd.read_csv('./data/file.csv')
newdf=pd.concat([df[df['Isval']==0].sample(frac=0.05,random_state=10),df[df['Isval']==1]])
newdf['labels']=1

In [ ]:
dls = db.dataloaders(source=newdf, bs=2, num_workers=8,drop_last=True,pin_memory=True)

In [ ]:
from fastai.distributed import *
dl1 = DistributedDL(dls, 0, 4)

In [ ]:
notebook2script(fname='./04_data.ipynb')

Converted 04_data.ipynb.
